<a href="https://colab.research.google.com/github/tejasvi1303/mentalhealthprediction/blob/main/Mental_Health_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install nltk joblib streamlit pyngrok --quiet

In [53]:
!sudo apt install tesseract-ocr -y
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [75]:
import pandas as pd
import numpy as np
import string
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import seaborn as sns
%matplotlib inline
import re
from sklearn.naive_bayes import BernoulliNB
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import joblib
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from multiprocessing import Pool, cpu_count
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras import Model
# Download necessary NLTK data files
# Download necessary NLTK data files
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)


import os

file_path = '/content/Combined Data.csv'
if os.path.exists(file_path):
    print("File found at:", file_path)
else:
    print("File not found.")


df = pd.read_csv("Combined Data.csv", encoding='latin1', on_bad_lines='skip')
df.head(10)
df.tail(10)
print(df.isnull().sum())
df = df.dropna()
print(df.isnull().sum())
df['status'].nunique()
sentiment_counts=df['status'].value_counts()
print(sentiment_counts)
df['status'].unique()
df.shape
df.info()
# Calculate the length of each statement
df['statement_length'] = df['statement'].apply(len)

# Display basic statistics of statement lengths
print(df['statement_length'].describe())

# Taking a sample of the dataframe, e.g., 20,000 rows
sample_size = 20000
df_sample = df.sample(n=sample_size, random_state=1)
# Load the spacy model
nlp = spacy.load("en_core_web_sm")
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)
def preprocess_texts(texts):
    return [preprocess_text(text) for text in texts]


# Split the data into batches for multiprocessing
num_cores = cpu_count()
df_split = np.array_split(df_sample, num_cores)

# Create a multiprocessing Pool
with Pool(num_cores) as pool:
    # Preprocess the text in parallel
    results = pool.map(preprocess_texts, [batch['statement'].tolist() for batch in df_split])

# Combine the results
df_sample['cleaned_statement'] = [item for sublist in results for item in sublist]

# Display the first few rows of the DataFrame to confirm the changes
print(df_sample[['statement', 'cleaned_statement']].head())
df_sample.head()
# Extract features and labels
processedtext = df_sample['cleaned_statement']
sentiment = df_sample['status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(processedtext, sentiment,
                                                    test_size=0.05, random_state=0)

print(f'X_train size: {len(X_train)}')
print(f'X_test size: {len(X_test)}')
print(f'y_train size: {len(y_train)}')
print(f'y_test size: {len(y_test)}')

vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)
print(f'Vectoriser fitted.')
print('No. of feature_words: ', len(vectoriser.get_feature_names_out()))

X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

# Use SMOTE to oversample the training data
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the parameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]
}

# Initialize the model
bnb = BernoulliNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=bnb, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and best score
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Score: {grid_search.best_score_}')

# Train the model with the best parameters
best_bnb = grid_search.best_estimator_
best_bnb.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_best_bnb = best_bnb.predict(X_test)

# Evaluate the model
print("Tuned Bernoulli Naive Bayes")
print(classification_report(y_test, y_pred_best_bnb))

File found at: /content/Combined Data.csv
Unnamed: 0      0
statement     362
status          0
dtype: int64
Unnamed: 0    0
statement     0
status        0
dtype: int64
status
Normal                  16343
Depression              15404
Suicidal                10652
Anxiety                  3841
Bipolar                  2777
Stress                   2587
Personality disorder     1077
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 52681 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  52681 non-null  int64 
 1   statement   52681 non-null  object
 2   status      52681 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB
count    52681.000000
mean       578.718153
std        846.274948
min          2.000000
25%         80.000000
50%        317.000000
75%        752.000000
max      32759.000000
Name: statement_length, dtype: float64


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


                                               statement  \
39488  i m and i have bad anxiety debilitating i have...   
1480   Jessica starred in the musical "Legally Blonde...   
47948  Im so tired I just dont see a point to my su...   
19681  My life 1 year ago was completely different. I...   
37629  RT @no_onespecixl: Know one enjoys my company ...   

                                       cleaned_statement  
39488  bad anxiety debilitating able keep job since w...  
1480   jessica starred musical legally blonde elle wo...  
47948  im tired dont see point suffering dont unde...  
19681  life 1 year ago completely different chick mag...  
37629  rt noonespecixl know one enjoys company make e...  
X_train size: 19000
X_test size: 1000
y_train size: 19000
y_test size: 1000
Vectoriser fitted.
No. of feature_words:  500000
Best Parameters: {'alpha': 0.1}
Best Score: 0.8011026708183809
Tuned Bernoulli Naive Bayes
                      precision    recall  f1-score   support

      

In [85]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming y_test are true labels and y_pred are predicted labels
accuracy = accuracy_score(y_test, y_pred_best_bnb)  # Replace y_pred_best_bnb with your prediction variable
print("Accuracy:", accuracy)

# Optional: print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best_bnb))

print("\n Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_best_bnb))

Accuracy: 0.573

Classification Report:
                      precision    recall  f1-score   support

             Anxiety       0.86      0.32      0.47        75
             Bipolar       1.00      0.09      0.17        55
          Depression       0.54      0.52      0.53       284
              Normal       0.56      0.98      0.71       310
Personality disorder       1.00      0.23      0.37        22
              Stress       1.00      0.10      0.18        50
            Suicidal       0.60      0.40      0.48       204

            accuracy                           0.57      1000
           macro avg       0.79      0.38      0.42      1000
        weighted avg       0.64      0.57      0.53      1000


 Confusion Matrix:
[[ 24   0  14  34   0   0   3]
 [  0   5  36  14   0   0   0]
 [  2   0 148  89   0   0  45]
 [  0   0   4 304   0   0   2]
 [  0   0  11   6   5   0   0]
 [  2   0  13  26   0   5   4]
 [  0   0  50  72   0   0  82]]


In [86]:
%%writefile mentalHealth.py
# Import necessary libraries
import streamlit as st
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from PIL import Image
import pytesseract
import nltk
from wordcloud import WordCloud
from collections import Counter
import io
import spacy
from multiprocessing import Pool, cpu_count

import nltk
nltk.data.path.clear()  # Clear any cached paths
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# Load spaCy model
@st.cache_resource
def load_spacy_model():
    return spacy.load("en_core_web_sm")

nlp = load_spacy_model()

# Preprocessing function
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Original NLTK preprocessing
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Enhanced preprocessing with spaCy
def enhanced_preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Process with spaCy
    doc = nlp(text)

    # Combine spaCy and NLTK for optimal results
    tokens = []
    for token in doc:
        if not token.is_stop and not token.is_punct and len(token.text.strip()) > 0:
            # Use spaCy's lemma but fall back to NLTK for pronouns
            lemma = token.lemma_ if token.lemma_ != '-PRON-' else lemmatizer.lemmatize(token.text)
            tokens.append(lemma)

    return ' '.join(tokens)

def preprocess_texts(texts):
    return [enhanced_preprocess_text(text) for text in texts]

# Function for parallel processing
def parallel_preprocessing(df, column_name):
    num_cores = cpu_count()
    df_split = np.array_split(df, num_cores)

    # Create a multiprocessing Pool
    with Pool(num_cores) as pool:
        # Preprocess the text in parallel
        results = pool.map(preprocess_texts, [batch[column_name].tolist() for batch in df_split])

    # Combine the results
    df['cleaned_statement'] = [item for sublist in results for item in sublist]
    return df

# OCR function to extract text from images
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

# Function to generate word cloud
def generate_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(title)
    ax.axis('off')
    return fig

# Function to extract mental health entities
def extract_mental_health_entities(text):
    doc = nlp(text)
    entities = []

    # Mental health related terms
    mental_health_terms = ["anxiety", "depression", "stress", "worried", "sad",
                          "anxious", "panic", "fear", "mood", "emotion"]

    for token in doc:
        if token.lemma_.lower() in mental_health_terms:
            entities.append((token.text, "MENTAL_CONDITION"))

    return entities

# Streamlit App Title
st.title("Mental Health Prediction")

# Create tabs for different input methods
tab1, tab2, tab3 = st.tabs(["Upload CSV", "Image Input", "Text Input"])

with tab1:
    # Upload File
    uploaded_file = st.file_uploader("Upload CSV File", type=["csv"], key="csv_uploader")
    if uploaded_file:
        df = pd.read_csv(uploaded_file, index_col=0, encoding='latin1')
        # Limit the dataset to 5000 entries
        # if len(df) > 5000:
        #     df = df.sample(n=5000, random_state=42)

        # Display data info
        st.write("### Data Preview")
        st.write(df.head())

        # Check for missing values
        st.write("### Missing Values")
        st.write(df.isnull().sum())

        # Drop missing values
        df = df.dropna()
        st.write("After dropping missing values:", df.shape)

        # EDA section
        st.write("### Exploratory Data Analysis")
        st.write(df.info())
        st.write(df.describe())

        # Distribution of sentiments
        sentiment_counts = df['status'].value_counts()
        st.write("Sentiment Counts:")
        st.write(sentiment_counts)

        fig, ax = plt.subplots()
        sentiment_counts.plot(kind='bar', title='Distribution of Sentiments', ax=ax)
        st.pyplot(fig)

        # Calculate statement lengths
        df['statement_length'] = df['statement'].apply(len)

        # Statement length statistics before outlier removal
        st.write("### Statement Length Statistics (Before Outlier Removal)")
        st.write(df['statement_length'].describe())

        # Distribution of statement lengths before outlier removal
        fig = plt.figure()
        ax = fig.add_subplot(111)  # Ensure ax is bound to fig
        df['statement_length'].hist(bins=100, ax=ax)
        ax.set_title('Distribution of Statement Lengths (Before Outliers Removal)')
        ax.set_xlabel('Length of Statements')
        ax.set_ylabel('Frequency')
        st.pyplot(fig)

        # Compare statement lengths across categories using a boxplot before outlier removal
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.boxplot(data=df, x='status', y='statement_length', palette='coolwarm', ax=ax)
        ax.set_title('Statement Lengths by Mental Health Category (Before Outliers Removal)')
        ax.set_xlabel('Mental Health Category')
        ax.set_ylabel('Statement Length')
        plt.xticks(rotation=45)
        st.pyplot(fig)

        # Violin plot before outlier removal
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.violinplot(data=df, x='status', y='statement_length', palette='cool', split=True, inner="quartile", ax=ax)
        ax.set_title('Violin Plot of Statement Lengths Split by Category (Before Outliers Removal)')
        ax.set_xlabel('Mental Health Category')
        ax.set_ylabel('Statement Length')
        plt.xticks(rotation=45)
        st.pyplot(fig)

        # Remove outliers based on IQR
        Q1 = df['statement_length'].quantile(0.25)
        Q3 = df['statement_length'].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        filtered_df = df[(df['statement_length'] >= lower_bound) & (df['statement_length'] <= upper_bound)]

        st.write(f"Original dataset size: {len(df)} rows")
        st.write(f"After outlier removal: {len(filtered_df)} rows")
        st.write(f"Removed {len(df) - len(filtered_df)} outliers")

        # Visualizations after outlier removal
        st.write("### Visualizations After Outlier Removal")

        # Plot the distribution of statement lengths without outliers
        fig, ax = plt.subplots()
        filtered_df['statement_length'].hist(bins=100, ax=ax)
        ax.set_title('Distribution of Statement Lengths (After Outliers Removal)')
        ax.set_xlabel('Length of Statements')
        ax.set_ylabel('Frequency')
        st.pyplot(fig)

        # Understand the frequency of each mental health category
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.countplot(data=filtered_df, x='status', palette='Set2', ax=ax)
        ax.set_title('Distribution of Mental Health Categories')
        ax.set_xlabel('Mental Health Category')
        ax.set_ylabel('Count')
        plt.xticks(rotation=45)
        st.pyplot(fig)

        # Compare statement lengths across categories using a boxplot after outlier removal
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.boxplot(data=filtered_df, x='status', y='statement_length', palette='coolwarm', ax=ax)
        ax.set_title('Statement Lengths by Mental Health Category (After Outlier Removal)')
        ax.set_xlabel('Mental Health Category')
        ax.set_ylabel('Statement Length')
        plt.xticks(rotation=45)
        st.pyplot(fig)

        # Violin plot after outlier removal
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.violinplot(data=filtered_df, x='status', y='statement_length', palette='cool', inner="quartile", ax=ax)
        ax.set_title('Violin Plot of Statement Lengths by Mental Health Category (After Outlier Removal)')
        ax.set_xlabel('Mental Health Category')
        ax.set_ylabel('Statement Length')
        plt.xticks(rotation=45)
        st.pyplot(fig)

        # Show the distribution of statement lengths by category
        fig, ax = plt.subplots(figsize=(8, 6))
        for category in filtered_df['status'].unique():
            sns.kdeplot(filtered_df[filtered_df['status'] == category]['statement_length'], label=category, ax=ax)
        ax.set_title('Density Plot of Statement Lengths by Category')
        ax.set_xlabel('Statement Length')
        ax.set_ylabel('Density')
        ax.legend()
        st.pyplot(fig)

        # Visualize word frequency across categories using a heatmap
        vectorizer = CountVectorizer(max_features=20, stop_words='english')
        X = vectorizer.fit_transform(df['statement'])
        word_freq_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
        word_freq_df['status'] = df['status']

        fig, ax = plt.subplots(figsize=(12, 6))
        sns.heatmap(word_freq_df.groupby('status').mean(), cmap='YlGnBu', annot=True, ax=ax)
        ax.set_title('Heatmap of Average Word Frequency by Mental Health Category')
        st.pyplot(fig)

        # Word clouds for each category
        st.write("### Word Clouds by Category")
        for status in filtered_df['status'].unique():
            status_text = ' '.join(filtered_df[filtered_df['status'] == status]['statement'])
            fig = generate_word_cloud(status_text, title=f'Word Cloud for {status}')
            st.pyplot(fig)

        # Add a progress bar for feedback
        progress_bar = st.progress(0)

        # Choose preprocessing method
        preprocessing_method = st.radio(
            "Select preprocessing method:",
            ("NLTK (Original)", "spaCy Enhanced", "Combined (NLTK + spaCy)")
        )

        # Add a confirm button
        confirm_preprocessing = st.button("Confirm Preprocessing Method")

        # Only proceed with preprocessing when the confirm button is clicked
        if confirm_preprocessing:
            # Preprocessing
            @st.cache_data
            def preprocess_statements(df, method):
                if method == "NLTK (Original)":
                    df['cleaned_statement'] = df['statement'].apply(preprocess_text)
                elif method == "spaCy Enhanced":
                    st.info("Using spaCy for preprocessing (may take longer but provides better linguistic analysis)")
                    df = parallel_preprocessing(df, 'statement')
                else:  # Combined
                    st.info("Using combined NLTK and spaCy preprocessing")
                    df['cleaned_statement'] = df['statement'].apply(enhanced_preprocess_text)
                return df
            filtered_df = preprocess_statements(filtered_df, preprocessing_method)
            progress_bar.progress(30)

            # Train-Test Split
            X = filtered_df['cleaned_statement']
            y = filtered_df['status']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            # TF-IDF Vectorization
            @st.cache_data
            def vectorize_data(X_train, X_test):
                vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=3000)
                X_train_tfidf = vectorizer.fit_transform(X_train)
                X_test_tfidf = vectorizer.transform(X_test)
                return vectorizer, X_train_tfidf, X_test_tfidf

            vectorizer, X_train_tfidf, X_test_tfidf = vectorize_data(X_train, X_test)
            progress_bar.progress(60)

            # SMOTE for Balancing
            smote = SMOTE(random_state=0)
            X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)
            progress_bar.progress(80)

            # Train SVM Model
            svm_model = SVC(kernel='linear', C=1.0, probability=True)
            svm_model.fit(X_train_resampled, y_train_resampled)
            progress_bar.progress(100)

            # Classification Report
            y_pred_svm = svm_model.predict(X_test_tfidf)
            st.write("### Classification Report")
            st.text(classification_report(y_test, y_pred_svm))

            # Save model and vectorizer in session state
            st.session_state['model'] = svm_model
            st.session_state['vectorizer'] = vectorizer
            st.session_state['classes'] = svm_model.classes_
            st.session_state['preprocessing_method'] = preprocessing_method

            st.success(f"Model trained using {preprocessing_method} preprocessing method!")
        else :
            st.info("Please select a preprocessing method and click 'Confirm Preprocessing Method' to proceed with training.")

with tab2:
    st.write("### Upload an Image with Text")
    uploaded_image = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"], key="image_uploader")

    if uploaded_image is not None:
        # Display the uploaded image
        image = Image.open(uploaded_image)
        st.image(image, caption="Uploaded Image", use_column_width=True)

        # Extract text from the image
        if st.button("Extract Text and Classify"):
            with st.spinner("Extracting text from image..."):
                extracted_text = extract_text_from_image(image)

                if extracted_text.strip():
                    st.write("### Extracted Text:")
                    st.write(extracted_text)

                    # Check if model is trained
                    if 'model' in st.session_state and 'vectorizer' in st.session_state:
                        # Process and classify the extracted text
                        if st.session_state.get('preprocessing_method') == "spaCy Enhanced":
                            processed_text = enhanced_preprocess_text(extracted_text)
                        elif st.session_state.get('preprocessing_method') == "Combined (NLTK + spaCy)":
                            processed_text = enhanced_preprocess_text(extracted_text)
                        else:  # NLTK (Original)
                            processed_text = preprocess_text(extracted_text)


                        vectorized_text = st.session_state['vectorizer'].transform([processed_text])
                        prediction = st.session_state['model'].predict(vectorized_text)[0]
                        probabilities = st.session_state['model'].predict_proba(vectorized_text)[0]

                        st.write("### Classification Results:")
                        st.write(f"*Predicted Category:* {prediction}")
                        st.write("Confidence Scores:", dict(zip(st.session_state['classes'], probabilities)))

                        # Extract mental health entities
                        entities = extract_mental_health_entities(extracted_text)
                        if entities:
                            st.write("### Mental Health Entities Detected:")
                            for entity, label in entities:
                                st.write(f"- {entity} ({label})")
                        # Word cloud visualization for extracted text
                        if len(extracted_text.split()) > 3:
                            st.write("### Word Cloud:")
                            fig = generate_word_cloud(extracted_text, "Extracted Text Word Cloud")
                            st.pyplot(fig)

                    else:
                        st.warning("Please upload a CSV file in the 'Upload CSV' tab first to train the model.")
                else:
                    st.error("No text could be extracted from the image. Please try another image.")

with tab3:
    st.write("### Predict Mental Health Category")
    user_input = st.text_area("Enter a statement:")
    if st.button("Classify", key="classify_text"):
        if user_input:
            if 'model' in st.session_state and 'vectorizer' in st.session_state:
                # Use the same preprocessing method as training
                if st.session_state.get('preprocessing_method') == "spaCy Enhanced":
                    processed_text = enhanced_preprocess_text(user_input)
                elif st.session_state.get('preprocessing_method') == "Combined (NLTK + spaCy)":
                    processed_text = enhanced_preprocess_text(user_input)
                else:  # NLTK (Original)
                    processed_text = preprocess_text(user_input)

                vectorized_text = st.session_state['vectorizer'].transform([processed_text])
                prediction = st.session_state['model'].predict(vectorized_text)[0]
                probabilities = st.session_state['model'].predict_proba(vectorized_text)[0]

                st.write(f"*Predicted Category:* {prediction}")
                st.write("Confidence Scores:", dict(zip(st.session_state['classes'], probabilities)))

                # Extract mental health entities
                entities = extract_mental_health_entities(user_input)
                if entities:
                    st.write("### Mental Health Entities Detected:")
                    for entity, label in entities:
                        st.write(f"- {entity} ({label})")
                        # Word cloud visualization for user input
                if len(user_input.split()) > 3:
                    st.write("### Word Cloud:")
                    fig = generate_word_cloud(user_input, "Input Text Word Cloud")
                    st.pyplot(fig)

            else:
                st.warning("Please upload a CSV file in the 'Upload CSV' tab first to train the model.")

Overwriting mentalHealth.py


In [88]:
!killall ngrok

ngrok: no process found


In [89]:
from pyngrok import ngrok

# Set your ngrok auth token
ngrok.set_auth_token("2vNCVFg87afyBqt4Z8uJb5MhrDC_3HqemMVB9UZt4g7dEyyFM")

# Launch Streamlit in the background
!streamlit run mentalHealth.py &>/dev/null &

# Create tunnel and get public URL
public_url = ngrok.connect(addr=8501)
print(f"🚀 Streamlit app is live at: {public_url}")

🚀 Streamlit app is live at: NgrokTunnel: "https://8efcb21eb89b.ngrok-free.app" -> "http://localhost:8501"


In [74]:
# User input
user_text = input(" Enter a mental health-related statement: ")

# Preprocess the input
def preprocess_single_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

cleaned_input = preprocess_single_text(user_text)

# Transform the input using the same TF-IDF vectorizer
vectorized_input = vectoriser.transform([cleaned_input])

# Predict using the trained model
prediction = best_bnb.predict(vectorized_input)[0]

print(f"\n🔮 Predicted Mental Health Category: **{prediction}**")


 Enter a mental health-related statement: I am hoping it is soon. Before my birthday soon. I have completely given up on happiness, I have given up on trying to find friends, and I have given up on love. I have no reason to be here anymore, and I am tired of trying to convince myself otherwise. I am tired of false hope taunting me, always just out of my reach. Then, when I inevitable lose it, I end up feeling worse than before. It happens too often. I honestly might try filling my car with carbon monoxide tonight. I hear it can be peaceful. My life will end in suicide.

🔮 Predicted Mental Health Category: **Suicidal**
